# TODO:
1. Sampling from large data set for prototype
2. Writeup

In [1]:
# Importing Libraries
import os
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

# Loading custom built functions
from model.nearest_neighbor_model import KNN
from model.lightfm_model import lightfm_model
from model.baseline_model import baseline_bias_model
from model.als_model import get_best_rank, cross_validation, plot_performance_als
from utils.data_loader import load_spark_df, load_pandas_df, spark_to_sparse, sample_data_frame

from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from surprise.prediction_algorithms.knns import KNNWithZScore

## 1. Loading Data 

We implement a function to cache and load the dataframe from secondary memory to reduce data load time

In [2]:
# Setting Directory path
base_path = os.getcwd()
dir_name = 'ml-latest-small'
CACHE_DIR = base_path + '/cache/'
DATA_DIR =  base_path + '/data/'

# Loading the Data Frames
movies_spark_df = load_spark_df(dir_name=dir_name, 
                                file_name='movies', 
                                use_cache=True,
                                DATA_DIR=DATA_DIR,
                                CACHE_DIR=CACHE_DIR
                               )

ratings_spark_df = load_spark_df(dir_name=dir_name, 
                                 file_name='ratings', 
                                 use_cache=True,
                                 DATA_DIR=DATA_DIR,
                                 CACHE_DIR=CACHE_DIR)
print('\nSampling DataFrame ...')
ratings_spark_df = sample_data_frame(ratings_spark_df, ratio=0.1, min_user_threshold=5, min_item_threshold=5)

Loading from P:\rmahajan14\columbia\fall 2019\Personalization\project_1\personalization_1/cache/ml-latest-small_movies.msgpack
Loading from P:\rmahajan14\columbia\fall 2019\Personalization\project_1\personalization_1/cache/ml-latest-small_ratings.msgpack

Sampling DataFrame ...
Length before sampling: 100836
Length after sampling: 10094
Length after thresholding: 10008


There are a number of ways we could sample our data. In particular, we implemented and tested 3 sampling methods:

The first is raw random sampling, where we simply select a random sample of our data. It is a simple samplin g method, however, we found that it doesn't work well in practice, as we have no thresholding criteria - So by selecting random rows, we are not fine tuning the kind of users, or movies (popular or unpopuolar) we want to sample. Had the users and movies been uniformaly distributed, this would have been okay. But, we see that in our data set, the kinds of users and movies show a lot of hetrogenity, with the number of movies rated by each user, as well as the number of times each movie was rated, varying a lot across users. Therefore, we moved on to more sophisticated sampling technique

In the second sampling technique, we first selected a subset of the data based on a threshold. So we initially removed any user who had rated less than 'x' items, as well as any movie which had been rated less than 'y' times. We set x and y to 5. After getting the initial thresholded dataset, we then randomly sampled a fraction of 0.1. 

In the third technique, we first selected the rows with the top 'a' users, and the rows with the top 'b' movies. After getting those 2 datasets, we performed an inner merge to include only users which are in the top 'a', and movies which are in the top 'b'. From this merged dataset containing the most popular users and movies, we then sampled a fraction of the rows for our final sampled dataset.

Having tried all these sampling methods (detailed code in utils/sample.py), we selected method 2 as our final sampling technique. This was because choosing all users/movies abive a certain threshold was more in line with our final objective of building a recommendation system which caters to the needs of everyone, not just users who rate a lot of movies.

## 2. Analysis of methods

### 2.1 Baseline Method: Bias based model

We first fit a bias only model to the data to set a benchmark for baseline model. 

In [ ]:
%%time

baseline_bias_model(ratings_spark_df)

Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9335  0.9478  0.9470  0.9464  0.9520  0.9453  0.0062  
MAE (testset)     0.7309  0.7411  0.7490  0.7397  0.7429  0.7407  0.0058  
Fit time          0.02    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


Wall time: 21 s


### 2.2 Model based method using Alternating Least Squares method

We build a Matrix Factorization model using ALS method, and iterate over different rank ranges to find the optimal rank

#### 2.2.1 Finding best hyperparameter setting using cross validation

In [ ]:
%%time

# Creating a Parameter Grid for ALS
model = ALS(userCol="userId",
                  itemCol="movieId",
                  ratingCol="rating",
                  coldStartStrategy="drop",
                  nonnegative=True)

paramGrid = ParamGridBuilder() \
            .addGrid(model.maxIter, [3]) \
            .addGrid(model.regParam, [0.01,0.1]) \
            .addGrid(model.rank, [64, 128]) \
            .build()

# Finding best parameter combination from cross validation
best_hyper_parameter, best_model = cross_validation(ratings_spark_df, 
                                                     model=model, 
                                                     evaluator='Regression', 
                                                     param_grid=paramGrid, 
                                                     k_folds=3)

print("Best Hyper-parameter combination for ALS Model:")
display(best_hyper_parameter)
print('\n')

RMSE of Best Model on Test Set: 1.2794
Best Hyper-parameter combination for ALS Model:


,HyperParameter,Value
0,maxIter,3
1,regParam,0.01
2,rank,128




Wall time: 5min 55s


#### 2.1.2 For different ranks, plotting RMSE and coverage on training and test set

In [ ]:
%%time

pow_two_max_rank = 8

ranks = [2**i for i in range(pow_two_max_rank+1)]

report_df = get_best_rank(ratings_spark_df, ranks=ranks)

plot_performance_als(report_df)
display(report_df)
print('\n')

We observe the following:
1. The training error keeps on decreasing with increased rank, but the test error shows no significant improvement indicating signs of overfitting
2. The coverage of items improves with respect to rank
3. The time to fit the model takes expontially higher time in correlation with rank

Note: We use Catalog Coverage to take into account the number of unique movies that were recommended to atleast one user as a top choice amonsgt the set of all unique movies.

### 2.3 LightFM:

We use LightFM model to find how it performs over over dataset

In [ ]:
%%time

sparse_mat = spark_to_sparse(ratings_spark_df)
lightfm_model(sparse_mat, prec_at_k=10, train_split=0.8)

### 2.4. Neighborhood based method using Nearest Neighbor

We use Nearest Neighbor algorithm with z-score normalization of each user

In [ ]:
%%time

# Defining parameters for Nearest Neighbor model
sim_options = {'name': 'cosine',
               'user_based': True
               }
model = KNNWithZScore(sim_options=sim_options)

KNN(model=model, df=ratings_spark_df)

#### We observe that the Baseline Bias model performs quite well, and other more sophesiticated models (except lightFM) don't yield significant improvments over it hence the Bias model might be the most suited for production

#### We observe that LightFm model has high AUC, meaning it is producing quantifiably quality results.